In [1]:
 !pip install tables==3.6.1
 !pip install socceraction==0.2.0
 !nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.3 MB 4.9 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.7.0
    Uninstalling tables-3.7.0:
      Successfully uninstalled tables-3.7.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 7.5 MB/s 
  Created wheel for socceraction: filename=socceraction-0.2.0-py3-none-any.whl size=30656 sha256=a93b0ff05a618d8f5fefff9066d8de2b1832ba9f054541712befce4b4fbaf334
  Stored in directory: /root/.cache/pip/wheels/ad/b6/3c/8ae71bd96f4cc8ae86a4ee1fdbd7a6e29fc1e59cec98a543a6
Successfully built socceraction
Wed Sep  7 02:13:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------

In [2]:
from ast import literal_eval
import warnings

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import brier_score_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

import socceraction.vaep.features as features
import socceraction.vaep.labels as labels
from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

In [3]:
import warnings
from io import BytesIO
from pathlib import Path
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile

import pandas as pd
import socceraction.vaep.features as features
import socceraction.vaep.labels as labels
from sklearn.metrics import brier_score_loss, roc_auc_score
from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

In [4]:
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [5]:
# This section downloads the Wyscout dataset, collects the required information about the match events, and converts the match events into the SPADL representation.

# Download the Wyscout dataset; (Not needed in Kaggle)
# Construct an HDF5 file named wyscout.h5 that contains the relevant information from the dataset;
# Convert the wyscout.h5 file into a spadl.h5 file that contains the same information in the SPADL representation.

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')
#%mkdir -p '/content/drive/My Drive/football/'
#%cd '/content/drive/My Drive/football/'

In [8]:
data_files = {
    'events': 'https://ndownloader.figshare.com/files/14464685',  # ZIP file containing one JSON file for each competition
    'matches': 'https://ndownloader.figshare.com/files/14464622',  # ZIP file containing one JSON file for each competition
    'players': 'https://ndownloader.figshare.com/files/15073721',  # JSON file
    'teams': 'https://ndownloader.figshare.com/files/15073697'  # JSON file
}

In [9]:
for url in tqdm(data_files.values()):
    url_s3 = urlopen(url).geturl()
    path = Path(urlparse(url_s3).path)
    file_name = path.name
    file_local, _ = urlretrieve(url_s3, file_name)
    if is_zipfile(file_local):
        with ZipFile(file_local) as zip_file:
            zip_file.extractall()

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')

**in a HDF5 file, what we call "directories" or "folders" on our computers, are called groups and what we call files on our computer are called datasets.**

**TEAMS**


In [11]:
json_teams = read_json_file('teams.json')
df_teams = pd.read_json(json_teams)

In [12]:
df_teams.head(5)

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alavés,696,Deportivo Alavés,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club


In [13]:
 df_teams.to_hdf('wyscout.h5', key='teams', mode='w')

**PLAYERS**

 read the players.json file into a DataFrame object and store that object in the wyscout.h5 HDF5 file under the key players.

In [14]:
json_players = read_json_file('players.json')
df_players = pd.read_json(json_players)

In [15]:
df_players.head(2)

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423


In [16]:
df_players.to_hdf('wyscout.h5', key='players', mode='a')

**Matches**

Uncomment the competitions that you want to include in your dataset.

In [18]:
competitions = [
     'England',
   #  'France',
   #  'Germany',
   #  'Italy',
   # 'Spain',
   'European Championship',
#   'World Cup'
]

read the matches.json files for the selected competitions into a DataFrame object and store that object in the wyscout.h5 HDF5 file under the key matches.

In [19]:
dfs_matches = []
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_matches = f'matches_{competition_name}.json'
    json_matches = read_json_file(file_matches)
    df_matches = pd.read_json(json_matches)
    dfs_matches.append(df_matches)
df_matches = pd.concat(dfs_matches)

In [20]:
df_matches.head(3)

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId,groupName
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364,NaN
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364,NaN
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364,NaN


In [21]:
df_matches.to_hdf('wyscout.h5', key='matches', mode='a')

**EVENTS**

read the events.json files for the selected competitions into a DataFrame object and store that object in the wyscout.h5 HDF5 file under the key events/match_match-id

In [22]:
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_events = f'events_{competition_name}.json'
    json_events = read_json_file(file_events)
    df_events = pd.read_json(json_events)
    df_events_matches = df_events.groupby('matchId', as_index=False)
    for match_id, df_events_match in df_events_matches:
        df_events_match.to_hdf('wyscout.h5', key=f'events/match_{match_id}', mode='a')

**Convert the Wyscout data to the SPADL representation**

The following cell calls the convert_to_spadl function from the socceraction library to convert the wyscout.h5 HDF5 file into the spadl.h5 HDF5 file.

In [23]:
convert_to_spadl('wyscout.h5', 'spadl.h5')

...Inserting actiontypes
...Inserting bodyparts
...Inserting results
...Converting games
...Converting players
...Converting teams
...Generating player_games


100%|██████████| 431/431 [00:36<00:00, 11.78game/s]


...Converting events to actions


100%|██████████| 431/431 [04:05<00:00,  1.75game/s]


**Value game states**

1-Generate the features to describe the game states;

2-Generate the labels that capture the value of the game states;

3-Compose a dataset by selecting a set of features and the labels of the game states;

4-Train predictive machine learning models using the dataset;

5-Value the game states using the trained predictive machine learning model.

In [24]:
df_games = pd.read_hdf('spadl.h5', key='games')
df_actiontypes = pd.read_hdf('spadl.h5', key='actiontypes')
df_bodyparts = pd.read_hdf('spadl.h5', key='bodyparts')
df_results = pd.read_hdf('spadl.h5', key='results')

In [25]:
nb_prev_actions = 3

**Generate game state features**

The following cell lists a number of feature generators from the features module in the socceraction library. Each function expects either a DataFrame object containing actions (i.e., individual actions) or a list of DataFrame objects containing consecutive actions (i.e., game states), and returns the corresponding feature for the individual action or game state.

In [26]:
functions_features = [
    features.actiontype_onehot,
    features.bodypart_onehot,
    features.result_onehot,
    features.goalscore,
    features.startlocation,
    features.endlocation,
    features.movement,
    features.space_delta,
    features.startpolar,
    features.endpolar,
    features.team,
    features.time_delta
]

The following cell generates game states from consecutive actions in each game and computes the features for each game state.

1- Obtain the actions for the game (i.e., df_actions) by looping through the games;

2- Construct game states of a given length from the actions (i.e., dfs_gamestates);

3- Compute the features for the constructed game states (i.e., df_features) by looping through the list of feature generators.

In [27]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    dfs_gamestates = features.gamestates(df_actions, nb_prev_actions=nb_prev_actions)
    dfs_gamestates = features.play_left_to_right(dfs_gamestates, game['home_team_id'])
    
    df_features = pd.concat([function(dfs_gamestates) for function in functions_features], axis=1)
    df_features.to_hdf('features.h5', key=f'game_{game_id}')

  0%|          | 0/431 [00:00<?, ?it/s]

**Generate game state labels**

The following cell lists a number of label generators from the labels module in the socceraction library. Each function expects either a DataFrame object containing actions (i.e., individual actions) or a list of DataFrame objects containing consecutive actions (i.e., game states), and returns the corresponding label for the individual action or game state.

In [28]:
functions_labels = [
    labels.scores,
    labels.concedes
]

The following cell computes the labels for each action:

1- Obtain the actions for the game (i.e., df_actions) by looping through the games;

2- Compute the labels for the actions (i.e., df_labels) by looping through the list of label generators.

In [29]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    df_labels = pd.concat([function(df_actions) for function in functions_labels], axis=1)
    df_labels.to_hdf('labels.h5', key=f'game_{game_id}')

  0%|          | 0/431 [00:00<?, ?it/s]

**Generate dataset**

The following cell generates a list of names for the features to be included in the dataset.

In [30]:
columns_features = features.feature_column_names(functions_features, nb_prev_actions=nb_prev_actions)

The following cell obtains the relevant features for each game and stores them in the df_features DataFrame object.

In [31]:
dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    dfs_features.append(df_features[columns_features])
df_features = pd.concat(dfs_features).reset_index(drop=True)

  0%|          | 0/431 [00:00<?, ?it/s]

In [32]:
df_features.head(5)

,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,type_take_on_a0,type_foul_a0,type_tackle_a0,...,end_dist_to_goal_a0,end_angle_to_goal_a0,end_dist_to_goal_a1,end_angle_to_goal_a1,end_dist_to_goal_a2,end_angle_to_goal_a2,team_1,team_2,time_delta_1,time_delta_2
0,True,False,False,False,False,False,False,False,False,False,...,63.091679,0.053916,63.091679,0.053916,63.091679,0.053916,True,True,0.000000,0.000000
1,True,False,False,False,False,False,False,False,False,False,...,68.328929,0.355773,63.091679,0.053916,63.091679,0.053916,True,True,1.997756,1.997756
2,True,False,False,False,False,False,False,False,False,False,...,73.715416,0.185556,68.328929,0.355773,63.091679,0.053916,True,True,0.771744,2.769500
3,True,False,False,False,False,False,False,False,False,False,...,38.707772,0.396818,73.715416,0.185556,68.328929,0.355773,True,True,2.174464,2.946208
4,True,False,False,False,False,False,False,False,False,False,...,37.425928,0.620467,38.707772,0.396818,73.715416,0.185556,True,True,3.907382,6.081846


The following cell lists the names of the labels to be included in the dataset.


In [33]:
columns_labels = [
    'scores',
    'concedes'
]

The following cell obtains the relevant labels for each game and stores them in the df_labels DataFrame object.

In [34]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    dfs_labels.append(df_labels[columns_labels])
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

  0%|          | 0/431 [00:00<?, ?it/s]

In [35]:
df_labels.head(5)

,scores,concedes
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False


**Train classifiers**

The following cell trains an XGBoost classifier for each label using the computed features. For each label:

1- Construct an XGBoost classifier with default hyperparameters;

2- Train the classifier using the computed features and the label;

3- Store the trained classifier in the models dict.

In [36]:
%%time
models = {}
for column_labels in columns_labels:
    model = XGBClassifier(
        eval_metric='logloss',
        use_label_encoder=False
    )
    model.fit(df_features, df_labels[column_labels])
    models[column_labels] = model

CPU times: user 6min 17s, sys: 2.03 s, total: 6min 19s
Wall time: 6min 16s


**Estimate probabilities**

The following cell predicts the labels for the game states using the trained XGBoost classifier. For each label:

1- Retrieve the model for the label;

2- Estimate the probabilities of the labels being False and True given the computed features;

3- Keep the probabilities for the True label;

4- Store the probabilities as a Series object in the dfs_predictions dict.

In [37]:
dfs_predictions = {}
for column_labels in columns_labels:
    model = models[column_labels]
    probabilities = model.predict_proba(df_features)
    predictions = probabilities[:, 1]
    dfs_predictions[column_labels] = pd.Series(predictions)
df_predictions = pd.concat(dfs_predictions, axis=1)

In [38]:
df_predictions.head(5)

,scores,concedes
0,0.006917,0.002082
1,0.006648,0.001784
2,0.005505,0.001959
3,0.011810,0.001875
4,0.014758,0.001582


The following cell obtains the game_id for each action in order to store the predictions per game.

In [39]:
dfs_game_ids = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    dfs_game_ids.append(df_actions['game_id'])
df_game_ids = pd.concat(dfs_game_ids, axis=0).astype('int').reset_index(drop=True)

  0%|          | 0/431 [00:00<?, ?it/s]

The following cell concatenates the DataFrame objects with predictions and game_ids for each action into a single DataFrame object.

In [40]:
df_predictions = pd.concat([df_predictions, df_game_ids], axis=1)

In [41]:
df_predictions.head(5)

,scores,concedes,game_id
0,0.006917,0.002082,2500089
1,0.006648,0.001784,2500089
2,0.005505,0.001959,2500089
3,0.011810,0.001875,2500089
4,0.014758,0.001582,2500089


The following cell groups the predictions per game based on their game_id.

In [42]:
df_predictions_per_game = df_predictions.groupby('game_id')

The following cell stores the predictions in the predictions.h5 HDF5 file per game.

In [43]:
for game_id, df_predictions in tqdm(df_predictions_per_game):
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions[columns_labels].to_hdf('predictions.h5', key=f'game_{game_id}')

  0%|          | 0/431 [00:00<?, ?it/s]

**Value on-the-ball actions**

Note: The code in this section is based on the 4-compute-vaep-values.ipynb notebook in the socceraction repository.

In [44]:
df_players = pd.read_hdf('spadl.h5', key='players')
df_teams = pd.read_hdf('spadl.h5', key='teams')

In [45]:
dfs_values = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .merge(df_players, how='left')
        .merge(df_teams, how='left')
        .reset_index(drop=True)
    )
    
    df_predictions = pd.read_hdf('predictions.h5', key=f'game_{game_id}')
    df_values = value(df_actions, df_predictions['scores'], df_predictions['concedes'])
    
    df_all = pd.concat([df_actions, df_predictions, df_values], axis=1)
    dfs_values.append(df_all)

  0%|          | 0/431 [00:00<?, ?it/s]

In [46]:
df_values = (pd.concat(dfs_values)
    .sort_values(['game_id', 'period_id', 'time_seconds'])
    .reset_index(drop=True)
)

In [47]:
df_values[
    ['short_name', 'scores', 'concedes', 'offensive_value', 'defensive_value', 'vaep_value']
].head(10)

,short_name,scores,concedes,offensive_value,defensive_value,vaep_value
0,O. Giroud,0.008321,0.002245,0.000000,-0.000000,0.000000
1,A. Griezmann,0.007174,0.002148,-0.001146,0.000097,-0.001049
2,N. Kanté,0.006253,0.002068,-0.000922,0.000080,-0.000842
3,L. Koscielny,0.005594,0.003471,-0.000658,-0.001403,-0.002062
4,P. Evra,0.007118,0.003616,0.007118,-0.003616,0.003502
5,C. Săpunaru,0.002930,0.022312,-0.000686,-0.015193,-0.015880
6,C. Săpunaru,0.003072,0.019959,0.000142,0.002353,0.002495
7,B. Matuidi,0.031174,0.006300,0.011215,-0.003228,0.007988
8,C. Tătărușanu,0.004676,0.044425,-0.001624,-0.013251,-0.014875
9,C. Tătărușanu,0.005320,0.021012,0.000643,0.023414,0.024057


**Rate players**

Note: The code in this section is based on the 5-top-players.ipynb notebook in the socceraction repository.

**Rate according to total VAEP value**

In [48]:
df_ranking = (df_values[['player_id', 'team_name', 'short_name', 'vaep_value']]
    .groupby(['player_id', 'team_name', 'short_name'])
    .agg(vaep_count=('vaep_value', 'count'), vaep_sum=('vaep_value', 'sum'))
    .sort_values('vaep_sum', ascending=False)
    .reset_index()
)

In [49]:
df_ranking.head(10)

,player_id,team_name,short_name,vaep_count,vaep_sum
0,120353.0,Liverpool FC,Mohamed Salah,1568,28.471001
1,8717.0,Tottenham Hotspur FC,H. Kane,1153,19.935841
2,26150.0,Leicester City FC,R. Mahrez,2022,18.114845
3,38021.0,Manchester City FC,K. De Bruyne,3528,17.757875
4,25707.0,Chelsea FC,E. Hazard,1974,16.549902
5,54.0,Tottenham Hotspur FC,C. Eriksen,2959,16.183369
6,8325.0,Manchester City FC,S. Agüero,982,14.148968
7,8317.0,Manchester City FC,David Silva,2925,13.746583
8,11066.0,Manchester City FC,R. Sterling,1693,13.702565
9,49872.0,Stoke City FC,X. Shaqiri,1722,13.280252


**Rate according to total VAEP value per 90 minutes**

In [50]:
df_player_games = pd.read_hdf('spadl.h5', 'player_games')
df_player_games = df_player_games[df_player_games['game_id'].isin(df_games['game_id'])]

In [51]:
df_minutes_played = (df_player_games[['player_id', 'minutes_played']]
    .groupby('player_id')
    .sum()
    .reset_index()
)

In [52]:
df_minutes_played.head(5)

,player_id,minutes_played
0,36,1701.689967
1,38,382.741398
2,48,3715.247229
3,54,3350.424005
4,56,267.266792


In [53]:
df_ranking_p90 = df_ranking.merge(df_minutes_played)
df_ranking_p90 = df_ranking_p90[df_ranking_p90['minutes_played'] > 360]
df_ranking_p90['vaep_rating'] = df_ranking_p90['vaep_sum'] * 90 / df_ranking_p90['minutes_played']
df_ranking_p90 = df_ranking_p90.sort_values('vaep_rating', ascending=False)

In [54]:
df_ranking_p90.head(5)

,player_id,team_name,short_name,vaep_count,vaep_sum,minutes_played,vaep_rating
0,120353.0,Liverpool FC,Mohamed Salah,1568,28.471001,2996.529579,0.855119
201,3682.0,France,A. Griezmann,347,4.816000,538.531925,0.804855
232,25714.0,France,D. Payet,438,4.236880,516.544493,0.738212
58,3802.0,Liverpool FC,Philippe Coutinho,1014,8.849359,1134.538547,0.701997
9,8325.0,Manchester City FC,S. Agüero,982,14.148968,2038.785220,0.624591


In [55]:
df_ranking_p90.to_csv('ranking.csv', index=False)

**END.**